In [1]:
import os
import pandas as pd
import numpy as np
from scipy.stats import iqr
from statsmodels.formula.api import ols
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
# Create bar plot
plt.style.use('seaborn-v0_8-poster')
#plt.style.use('seaborn-v0_8')
fontsize = 11
plt.rcParams.update({
    "font.size": fontsize,          # Global font size
    "axes.labelsize": fontsize,     # X and Y axis labels
    "axes.titlesize": fontsize,     # Title (if used)
    "xtick.labelsize": fontsize,    # X-axis tick labels
    "ytick.labelsize": fontsize,    # Y-axis tick labels
    "legend.fontsize": fontsize,    # Legend font size
})

In [2]:
project_dir = '/Users/xiaoqianxiao/projects/NARSAD'
SCR_dir = os.path.join(project_dir, 'EDR')
fMRI_file = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/roi_cope_results_all.csv')
df_fMRI = pd.read_csv(fMRI_file)
drug_file = os.path.join(SCR_dir, 'drug_order.csv')
ECR_file = os.path.join(SCR_dir, 'ECR.csv')
df_drug = pd.read_csv(drug_file)
df_drug['group'] = df_drug['subID'].apply(lambda x: 'Patients' if x.startswith('N1') else 'Controls')
df_ECR = pd.read_csv(ECR_file)
df_behav = df_drug.merge(df_ECR, how='left', left_on='subID', right_on='subID')
plot_dir = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/plots')

In [3]:
def rename_roi(roi):
    cleaned_roi = roi.replace('_mask', '').replace('_mask', '')
    sides = ['lh', 'rh']
    regions = ['hippocampus', 'amygdala', 'vmpfc', 'insula', 'ACC']  # Add more as needed
    
    side = None
    region = None
    for s in sides:
        if s in cleaned_roi:
            side = s
            break
    for r in regions:
        if r in cleaned_roi:
            region = r
            break
    if side and region:
        return f"{side}_{region}"
    else:
        return cleaned_roi

In [4]:
# Define directories
root_dir = '/Users/xiaoqianxiao/projects'
project_name = 'NARSAD'

sub_no_MRI_phase2 = ['N102', 'N208']
sub_no_MRI_phase3 = ['N102', 'N208', 'N120']

SCR_dir = os.path.join(root_dir, project_name, 'EDR')
drug_file = os.path.join(SCR_dir, 'drug_order.csv')
ECR_file = os.path.join(SCR_dir, 'ECR.csv')

# Load behavioral data
df_drug = pd.read_csv(drug_file)
df_drug['group'] = df_drug['subID'].apply(lambda x: 'Patients' if x.startswith('N1') else 'Controls')
df_ECR = pd.read_csv(ECR_file)
df_behav = df_drug.merge(df_ECR, how='left', left_on='subID', right_on='subID')

group_info_df_phase2 = df_behav.loc[~df_behav['subID'].isin(sub_no_MRI_phase2)]
group_info_df_phase3 = df_behav.loc[~df_behav['subID'].isin(sub_no_MRI_phase3)]

fMRI_file = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/roi_cope_results_all.csv')
df_fMRI = pd.read_csv(fMRI_file)
df_fMRI['roi_name'] = df_fMRI['ROI'].apply(rename_roi)
# Define the desired order for 'roi_name'
roi_order = ['lh_ACC', 'rh_ACC', 
             'lh_vmpfc','rh_vmpfc',
             'lh_insula','rh_insula', 
             'lh_hippocampus', 'rh_hippocampus', 
             'lh_amygdala', 'rh_amygdala'
]

df_fMRI['roi_name'] = pd.Categorical(df_fMRI['roi_name'], categories=roi_order, ordered=True)

df_fMRI_phase2 = df_fMRI.loc[df_fMRI['Task']=='phase2'].copy()
df_fMRI_phase3 = df_fMRI.loc[df_fMRI['Task']=='phase3'].copy()

In [5]:
df_fMRI_phase2['subID'] = df_fMRI_phase2['Subject'].apply(lambda x: group_info_df_phase2.iloc[x - 1]['subID'])
df_fMRI_phase3['subID'] = df_fMRI_phase3['Subject'].apply(lambda x: group_info_df_phase3.iloc[x - 1]['subID'])
df_phase2 = df_fMRI_phase2.merge(group_info_df_phase2, how='left', left_on='subID', right_on='subID')
df_phase2 = df_phase2[df_phase2['Gender'].isin(['Female', 'Male'])].reset_index(drop=True)
df_phase3 = df_fMRI_phase3.merge(group_info_df_phase3, how='left', left_on='subID', right_on='subID')
df_phase3 = df_phase3[df_phase3['Gender'].isin(['Female', 'Male'])].reset_index(drop=True)

In [6]:
df_current = df_phase3
contrasts_of_interest = [1,3]
contrasts_CSS = contrasts_of_interest[0]
contrasts_CSR = contrasts_of_interest[1]

In [7]:
results = []
from scipy.stats import ttest_rel
for roi in df_current['roi_name'].unique():
    for group in df_current['group'].unique():
        for gender in df_current['Gender'].unique():
            for drug in df_current['Drug'].unique():
                df_tmp = df_current.loc[(df_current['roi_name'] == roi) & 
                                        (df_current['group'] == group) & 
                                        (df_current['Gender'] == gender) &
                                        (df_current['Drug'] == drug) &
                                        (df_current['Contrast'].isin(contrasts_of_interest))].copy()
                df_pivot = df_tmp.pivot(index='Subject', columns='Contrast', values='Mean_COPE_Value')
                t_statistic, p_value = stats.ttest_rel(df_pivot[[contrasts_CSS]], df_pivot[[contrasts_CSR]])
                results.append({
                'roi': roi,
                'Group': group,
                'Drug': drug,
                'Gender': gender,
                't_stat': t_statistic[0],
                'p_value': p_value[0],
                'n': len(df_pivot)  # Sample size
            })
df_results = pd.DataFrame(results)
df_results['p_value_corrected'] = multipletests(df_results['p_value'], method='bonferroni')[1]
df_results.sort_values(by='p_value_corrected', ascending=True)

,roi,Group,Drug,Gender,t_stat,p_value,n,p_value_corrected
52,rh_amygdala,Controls,Placebo,Female,-3.495607,0.003947,14,0.315773
0,lh_hippocampus,Patients,Placebo,Female,-1.109406,0.288993,13,1.000000
57,rh_insula,Patients,Oxytocin,Female,-1.898917,0.076983,16,1.000000
56,rh_insula,Patients,Placebo,Female,0.103427,0.919333,13,1.000000
55,rh_amygdala,Controls,Oxytocin,Male,-0.601138,0.559932,12,1.000000
...,...,...,...,...,...,...,...,...
23,lh_amygdala,Controls,Oxytocin,Male,-0.540383,0.599703,12,1.000000
22,lh_amygdala,Controls,Placebo,Male,-1.563410,0.141964,14,1.000000
21,lh_amygdala,Controls,Oxytocin,Female,-0.063320,0.950648,12,1.000000
78,rh_hippocampus,Controls,Placebo,Male,1.884730,0.082018,14,1.000000


In [8]:
# extinction
results_list = []
drug_results_list = []
for roi in df_current['roi_name'].unique():
    print('extinction results for {}'.format(roi))
    # contrast no.2: CS+_safe - fixation; contrast no.3: CS+_reinf - fixation;no.4: CS+_safe - CS-; contrast no.6: CS+_reinf - CS-;
    df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
    df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
    df['fMRI'] = df['CSR'] - df['CSS']
    df['z_fMRI'] = 1 + zscore(df['fMRI'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
    
    df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
    
    model_ols = ols("z_fMRI ~ C(group) * C(Drug) * C(Gender) + Avoidance_centered + demo_age", data=df)
    result_ols = model_ols.fit()
    print(result_ols.summary())
    combined_col = f'{roi}'
        
    for effect_idx in result_ols.pvalues.index:
        p_value = result_ols.pvalues[effect_idx]
        results_list.append({
            'Condition': combined_col,
            'Effect': effect_idx,
            'p_value': p_value
        })
    conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI 
    for effect_idx in result_ols.params.index:
        if effect_idx == 'C(Drug)[T.Placebo]': 
            drug_results_list.append({
                'Effect': effect_idx,
                'ROI': roi,
                'b': result_ols.params[effect_idx],  # Coefficient
                'SE': result_ols.bse[effect_idx],    # Standard error
                't': result_ols.tvalues[effect_idx], # t-statistic
                'p': result_ols.pvalues[effect_idx],  # p-value
                'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                'CI_upper': conf_int.loc[effect_idx, 1]
            })

# Create DataFrame for drug effects
df_drug = pd.DataFrame(drug_results_list)
df_drug = df_drug.sort_values('ROI')
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                 -0.038
Method:                 Least Squares   F-statistic:                    0.5915
Date:                Fri, 21 Mar 2025   Prob (F-statistic):              0.801
Time:                        14:59:29   Log-Likelihood:                -140.44
No. Observations:                 101   AIC:                             300.9
Df Residuals:                      91   BIC:                             327.0
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------

,Condition,Effect,p_value
96,rh_hippocampus,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.041139
35,lh_ACC,C(group)[T.Patients]:C(Gender)[T.Male],0.094498
36,lh_ACC,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.095959
64,rh_amygdala,C(group)[T.Patients]:C(Drug)[T.Placebo],0.097465
37,lh_ACC,C(group)[T.Patients]:C(Drug)[T.Placebo]:C(Gend...,0.111867
...,...,...,...
22,lh_amygdala,C(Drug)[T.Placebo],0.959167
54,lh_insula,C(group)[T.Patients]:C(Drug)[T.Placebo],0.963776
8,lh_hippocampus,Avoidance_centered,0.977173
24,lh_amygdala,C(group)[T.Patients]:C(Drug)[T.Placebo],0.982642


for the entire dataset \
rh-ACC: Drug effect, group * drug and group * gender(marginal) \
Avoidance rh-amy (lh-amy, marginal)

In [9]:
# extinction
results_list = []
drug_results_list = []
for roi in df_current['roi_name'].unique():
    print('extinction results for {}'.format(roi))
    # contrast no.2: CS+_safe - fixation; contrast no.3: CS+_reinf - fixation;no.4: CS+_safe - CS-; contrast no.6: CS+_reinf - CS-;
    df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
    df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
    df['fMRI'] = df['CSR'] - df['CSS']
    df['z_fMRI'] = 1 + zscore(df['fMRI'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
    
    df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
    
    model_ols = ols("z_fMRI ~ C(group) * C(Drug) * C(Gender)", data=df)
    result_ols = model_ols.fit()
    print(result_ols.summary())
    combined_col = f'{roi}'
        
    for effect_idx in result_ols.pvalues.index:
        p_value = result_ols.pvalues[effect_idx]
        results_list.append({
            'Condition': combined_col,
            'Effect': effect_idx,
            'p_value': p_value
        })
    conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI 
    for effect_idx in result_ols.params.index:
        if effect_idx == 'C(Drug)[T.Placebo]': 
            drug_results_list.append({
                'Effect': effect_idx,
                'ROI': roi,
                'b': result_ols.params[effect_idx],  # Coefficient
                'SE': result_ols.bse[effect_idx],    # Standard error
                't': result_ols.tvalues[effect_idx], # t-statistic
                'p': result_ols.pvalues[effect_idx],  # p-value
                'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                'CI_upper': conf_int.loc[effect_idx, 1]
            })

# Create DataFrame for drug effects
df_drug = pd.DataFrame(drug_results_list)
df_drug = df_drug.sort_values('ROI')
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                 -0.021
Method:                 Least Squares   F-statistic:                    0.7045
Date:                Fri, 21 Mar 2025   Prob (F-statistic):              0.668
Time:                        14:59:33   Log-Likelihood:                -140.70
No. Observations:                 101   AIC:                             297.4
Df Residuals:                      93   BIC:                             318.3
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------

,Condition,Effect,p_value
78,rh_hippocampus,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.040126
30,lh_ACC,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.094361
52,rh_amygdala,C(group)[T.Patients]:C(Drug)[T.Placebo],0.115703
31,lh_ACC,C(group)[T.Patients]:C(Drug)[T.Placebo]:C(Gend...,0.122888
29,lh_ACC,C(group)[T.Patients]:C(Gender)[T.Male],0.133091
...,...,...,...
73,rh_hippocampus,C(group)[T.Patients],0.939646
35,rh_vmpfc,C(Gender)[T.Male],0.942962
20,lh_amygdala,C(group)[T.Patients]:C(Drug)[T.Placebo],0.992475
44,lh_insula,C(group)[T.Patients]:C(Drug)[T.Placebo],0.993742


In [10]:
# extinction in only placebo, healthy females
results = []
for roi in df_current['roi_name'].unique():
    print('extinction results for {}'.format(roi))
    df_tmp = df_current[(df_current['Drug'] == 'Placebo') &
                        (df_current['group'] == 'Controls') & 
                        (df_current['Gender'] == 'Female') & 
                        (df_current['roi_name'] == roi) & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
    df_pivot = df_tmp.pivot(index='Subject', columns='Contrast', values='Mean_COPE_Value')
    t_statistic, p_value = stats.ttest_rel(df_pivot[[contrasts_CSS]], df_pivot[[contrasts_CSR]])
    results.append({
    'roi': roi,
    't_stat': t_statistic[0],
    'p_value': p_value[0],
    'n': len(df_pivot)  # Sample size
})
df_results = pd.DataFrame(results)
df_results['p_value_corrected'] = multipletests(df_results['p_value'], method='bonferroni')[1]
df_results.sort_values(by='p_value_corrected', ascending=True)

extinction results for lh_hippocampus
extinction results for rh_ACC
extinction results for lh_amygdala
extinction results for lh_ACC
extinction results for rh_vmpfc
extinction results for lh_insula
extinction results for rh_amygdala
extinction results for rh_insula
extinction results for lh_vmpfc
extinction results for rh_hippocampus


,roi,t_stat,p_value,n,p_value_corrected
6,rh_amygdala,-3.495607,0.003947,14,0.039472
0,lh_hippocampus,-1.304336,0.214740,14,1.000000
1,rh_ACC,0.105422,0.917650,14,1.000000
2,lh_amygdala,-0.188525,0.853376,14,1.000000
3,lh_ACC,-1.204493,0.249871,14,1.000000
4,rh_vmpfc,-0.319175,0.754665,14,1.000000
5,lh_insula,-0.958316,0.355388,14,1.000000
7,rh_insula,1.070659,0.303799,14,1.000000
8,lh_vmpfc,-1.377077,0.191739,14,1.000000
9,rh_hippocampus,-0.903789,0.382550,14,1.000000


In [11]:
# extinction in only placebo
results_list = []
drug_results_list = []
for roi in df_current['roi_name'].unique():
    print('extinction results for {}'.format(roi))
    df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['roi_name'] == roi) & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
    df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
    df['fMRI'] = df['CSR'] - df['CSS']
    df['z_fMRI'] = 1 + zscore(df['fMRI'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
    
    df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
    
    model_ols = ols("z_fMRI ~ C(group) * C(Gender) + Avoidance_centered + demo_age", data=df)
    result_ols = model_ols.fit()
    print(result_ols.summary())
    conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI     
    for effect_idx in result_ols.pvalues.index:
        p_value = result_ols.pvalues[effect_idx]
        results_list.append({
            'ROI': roi,
            'Effect': effect_idx,
            'b': result_ols.params[effect_idx],  # Coefficient
                'SE': result_ols.bse[effect_idx],    # Standard error
                't': result_ols.tvalues[effect_idx], # t-statistic
                'p': result_ols.pvalues[effect_idx],  # p-value
                'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                'CI_upper': conf_int.loc[effect_idx, 1]
        })
    
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p', ascending=True)

extinction results for lh_hippocampus
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                 -0.033
Method:                 Least Squares   F-statistic:                    0.6852
Date:                Fri, 21 Mar 2025   Prob (F-statistic):              0.637
Time:                        14:59:55   Log-Likelihood:                -69.072
No. Observations:                  50   AIC:                             150.1
Df Residuals:                      44   BIC:                             161.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

,ROI,Effect,b,SE,t,p,CI_lower,CI_upper
37,rh_amygdala,C(group)[T.Patients],-1.089589,0.406585,-2.679852,0.010324,-1.909007,-0.270170
39,rh_amygdala,C(group)[T.Patients]:C(Gender)[T.Male],1.374275,0.576617,2.383340,0.021538,0.212179,2.536370
38,rh_amygdala,C(Gender)[T.Male],-0.787384,0.382277,-2.059723,0.045371,-1.557811,-0.016956
56,rh_hippocampus,C(Gender)[T.Male],-0.784918,0.398729,-1.968549,0.055325,-1.588505,0.018668
22,lh_ACC,Avoidance_centered,0.013476,0.007103,1.897244,0.064371,-0.000839,0.027792
57,rh_hippocampus,C(group)[T.Patients]:C(Gender)[T.Male],1.115349,0.601434,1.854482,0.070380,-0.096762,2.327460
20,lh_ACC,C(Gender)[T.Male],-0.651070,0.383180,-1.699122,0.096357,-1.423319,0.121179
17,lh_amygdala,demo_age,-0.031897,0.022214,-1.435924,0.158096,-0.076666,0.012872
52,lh_vmpfc,Avoidance_centered,0.010142,0.007271,1.394942,0.170036,-0.004511,0.024796
32,lh_insula,C(Gender)[T.Male],-0.555047,0.403957,-1.374026,0.176393,-1.369169,0.259075


In [12]:
# extinction in each group
results_list = []
drug_results_list = []
for group in df_current['group'].unique():
    for roi in df_current['roi_name'].unique():
        print(f'extinction results for {roi} {group}')
        df_tmp = df_current[(df_current['roi_name'] == roi) & 
                          (df_current['group'] == group) & 
                          (df_current['Contrast'].isin(contrasts_of_interest))].copy()
        
        df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
        
        df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], 
                        columns='CS_Type', 
                        values='Mean_COPE_Value').reset_index()
        
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender','Drug']] = df[['group', 'Gender','Drug']].astype('category')
        
        model_ols = ols("z_fMRI ~ C(Drug) * C(Gender) + Avoidance_centered + demo_age", data=df)
        #model_ols = ols("z_fMRI ~ C(Drug)", data=df)
        result_ols = model_ols.fit()
        print(result_ols.summary())
        
        combined_col = f'{roi}_{group}'
        
        for effect_idx in result_ols.pvalues.index:
            p_value = result_ols.pvalues[effect_idx]
            results_list.append({
                'Condition': combined_col,
                'Effect': effect_idx,
                'p_value': p_value
            })
        conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI 
        for effect_idx in result_ols.params.index:
            if effect_idx == 'C(Drug)[T.Placebo]': 
                drug_results_list.append({
                    'Effect': effect_idx,
                    'group': group,
                    'ROI': roi,
                    'b': result_ols.params[effect_idx],  # Coefficient
                    'SE': result_ols.bse[effect_idx],    # Standard error
                    't': result_ols.tvalues[effect_idx], # t-statistic
                    'p': result_ols.pvalues[effect_idx],  # p-value
                    'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                    'CI_upper': conf_int.loc[effect_idx, 1]
                })
    
df_drug = pd.DataFrame(drug_results_list)
df_drug = df_drug.sort_values('p', ascending=True)
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus Patients
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.071
Model:                            OLS   Adj. R-squared:                 -0.037
Method:                 Least Squares   F-statistic:                    0.6584
Date:                Fri, 21 Mar 2025   Prob (F-statistic):              0.657
Time:                        15:00:20   Log-Likelihood:                -67.721
No. Observations:                  49   AIC:                             147.4
Df Residuals:                      43   BIC:                             158.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

,Condition,Effect,p_value
99,rh_amygdala_Controls,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.040548
22,lh_ACC_Patients,Avoidance_centered,0.060846
88,rh_vmpfc_Controls,Avoidance_centered,0.062357
52,lh_vmpfc_Patients,Avoidance_centered,0.065654
117,rh_hippocampus_Controls,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.070813
...,...,...,...
73,lh_amygdala_Controls,C(Drug)[T.Placebo],0.971403
29,rh_vmpfc_Patients,demo_age,0.975207
45,rh_insula_Patients,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.980935
17,lh_amygdala_Patients,demo_age,0.985425


In [13]:
import pandas as pd
from scipy.stats import zscore, ttest_ind

desired_comparisons = [
    'Female_Oxytocin vs. Female_Placebo',
    'Male_Oxytocin vs. Male_Placebo'
]

t_test_results_list = []

for group in df_current['group'].unique():
    for roi in df_current['roi_name'].unique():
        df_tmp = df_current[(df_current['roi_name'] == roi) & 
                           (df_current['group'] == group) & 
                           (df_current['Contrast'].isin(contrasts_of_interest))].copy()
        
        df_tmp.loc[df_tmp['Contrast'] == contrasts_CSS, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast'] == contrasts_CSR, 'CS_Type'] = 'CSR'
        
        df = df_tmp.pivot(index=['subID', 'Drug', 'Gender', 'group', 'Avoidance', 'demo_age'], 
                          columns='CS_Type', values='Mean_COPE_Value').reset_index()
        
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])  # Offset of 1 assumed intentional
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')
        
        group_labels = df['Gender'].astype(str) + "_" + df['Drug'].astype(str)
        
        for comparison in desired_comparisons:
            g1, g2 = comparison.split(' vs. ')
            if g1 in group_labels.values and g2 in group_labels.values:  # Check if both groups exist
                group1_data = df[group_labels == g1]['z_fMRI']
                group2_data = df[group_labels == g2]['z_fMRI']
                
                t_stat, p_value = ttest_ind(group1_data, group2_data, nan_policy='omit')
                
                mean_diff = group1_data.mean() - group2_data.mean()
                se = ((group1_data.var() / group1_data.count()) + (group2_data.var() / group2_data.count())) ** 0.5
                
                t_test_results_list.append({
                    'ROI': roi,
                    'Comparison': comparison,
                    'Mean_Diff': mean_diff,
                    'SE': se,
                    't': t_stat,
                    'p': p_value
                })

# Create DataFrame with all results
df_t_test = pd.DataFrame(t_test_results_list)
df_t_test = df_t_test.sort_values('p', ascending=True)  # Sort by p-value

In [14]:
results_list = []
# extinction, in each gender
for g in df_current['Gender'].unique():
    for roi in df_current['roi_name'].unique():
        print(f'extinction results for {roi} {g}')
        df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Gender'] == g) & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
        df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
        df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
        
        model_ols = ols("z_fMRI ~ C(group) * C(Drug) + Avoidance_centered + demo_age", data=df)
        result_ols = model_ols.fit()
        print(result_ols.summary())
        combined_col = f'{roi}_{g}'
        
        for effect_idx in result_ols.pvalues.index:
            p_value = result_ols.pvalues[effect_idx]
            results_list.append({
                'Condition': combined_col,
                'Gender': g,
                'Effect': effect_idx,
                'p_value': p_value
            })
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus Female
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.045
Method:                 Least Squares   F-statistic:                     1.508
Date:                Fri, 21 Mar 2025   Prob (F-statistic):              0.205
Time:                        15:00:32   Log-Likelihood:                -74.105
No. Observations:                  55   AIC:                             160.2
Df Residuals:                      49   BIC:                             172.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------

,Condition,Gender,Effect,p_value
116,rh_hippocampus_Male,Male,C(Drug)[T.Placebo],0.007628
5,lh_hippocampus_Female,Female,demo_age,0.032539
101,rh_amygdala_Male,Male,demo_age,0.055590
47,rh_insula_Female,Female,demo_age,0.068978
68,rh_ACC_Male,Male,C(Drug)[T.Placebo],0.095103
...,...,...,...,...
14,lh_amygdala_Female,Female,C(Drug)[T.Placebo],0.977201
33,lh_insula_Female,Female,C(group)[T.Patients]:C(Drug)[T.Placebo],0.990005
15,lh_amygdala_Female,Female,C(group)[T.Patients]:C(Drug)[T.Placebo],0.990834
32,lh_insula_Female,Female,C(Drug)[T.Placebo],0.997138


in Gender specific data subset, \
for Female: \
    drug effect in rh-acc, \
    group * drug in rh-acc
    age in rh-hip \
    avoidance in lh-vmpfc \
for Male: \
    age in rh-vmpfc \
    group * drun in rh-amy \
    avoidance(marginal) in lh_amygdala

In [15]:
import pandas as pd 
from statsmodels.formula.api import ols
from scipy.stats import zscore

results_list = []
for g in df_current['Gender'].unique():
    for group in df_current['group'].unique():
        for roi in df_current['roi_name'].unique():
            print(f'extinction results for {roi} {g}')
            df_tmp = df_current[(df_current['roi_name'] == roi) & 
                              (df_current['Gender'] == g) & 
                              (df_current['group'] == group) & 
                              (df_current['Contrast'].isin(contrasts_of_interest))].copy()
            
            df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
            df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
            
            df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], 
                            columns='CS_Type', 
                            values='Mean_COPE_Value').reset_index()
            
            df['fMRI'] = df['CSR'] - df['CSS']
            df['z_fMRI'] = 1 + zscore(df['fMRI'])
            df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
            df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
            
            df[['group', 'Gender','Drug']] = df[['group', 'Gender','Drug']].astype('category')
            
            model_ols = ols("z_fMRI ~ C(Drug) + Avoidance_centered + demo_age", data=df)
            
            #model_ols = ols("z_fMRI ~ C(Drug)", data=df)
            result_ols = model_ols.fit()
            print(result_ols.summary())
            
            combined_col = f'{roi}_{group}_{g}'
            
            for effect_idx in result_ols.pvalues.index:
                p_value = result_ols.pvalues[effect_idx]
                results_list.append({
                    'Gender': g,
                    'Group': group,
                    'Condition': combined_col,
                    'Effect': effect_idx,
                    'p_value': p_value
                })
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus Female
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.067
Model:                            OLS   Adj. R-squared:                 -0.045
Method:                 Least Squares   F-statistic:                    0.5965
Date:                Fri, 21 Mar 2025   Prob (F-statistic):              0.623
Time:                        15:00:37   Log-Likelihood:                -40.147
No. Observations:                  29   AIC:                             88.29
Df Residuals:                      25   BIC:                             93.76
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

,Gender,Group,Condition,Effect,p_value
43,Female,Controls,lh_hippocampus_Controls_Female,demo_age,0.006520
157,Male,Controls,rh_hippocampus_Controls_Male,C(Drug)[T.Placebo],0.023497
138,Male,Controls,rh_vmpfc_Controls_Male,Avoidance_centered,0.047414
147,Male,Controls,rh_amygdala_Controls_Male,demo_age,0.077181
126,Male,Controls,rh_ACC_Controls_Male,Avoidance_centered,0.079802
...,...,...,...,...,...
3,Female,Patients,lh_hippocampus_Patients_Female,demo_age,0.965298
42,Female,Controls,lh_hippocampus_Controls_Female,Avoidance_centered,0.974106
49,Female,Controls,lh_amygdala_Controls_Female,C(Drug)[T.Placebo],0.978823
15,Female,Patients,lh_ACC_Patients_Female,demo_age,0.992347


In [16]:
# placebo data, gender * group
df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
df = df_tmp.copy()
df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()

df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')

plt.style.use('seaborn-v0_8-poster')

g = sns.catplot(x='Gender', y='z_fMRI', hue='CS_Type',
                row='roi_name', 
                col='group',
                kind='bar', 
                hue_order=['CSS', 'CSR'],
                errorbar='se',
                height=6, aspect=1,
                data=df)

g.set_axis_labels('', 'z_fMRI')
#g.legend.set_title('')
  
y_min = df['z_fMRI'].min() - 0.5
y_max = df['z_fMRI'].max() + 0.5
g.set(ylim=(0, 2.2))

g.figure.suptitle('', fontsize=22, y=1.01)  # Add title above plot

g.figure.set_dpi(300)
plt.savefig(f'{plot_dir}/reinstatement_placebodata_gender_group.png', bbox_inches='tight')
plt.close()
#plt.show()

In [17]:
# placebo data, gender * group
df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
df = df_tmp.copy()
df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()

df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')

plt.style.use('seaborn-v0_8-poster')

g = sns.catplot(x='Gender', y='z_fMRI', hue='CS_Type',
                row='roi_name', 
                kind='bar', 
                hue_order=['CSS', 'CSR'],
                errorbar='se',
                height=6, aspect=1,
                data=df)

g.set_axis_labels('', 'z_fMRI')
#g.legend.set_title('')
  
y_min = df['z_fMRI'].min() - 0.5
y_max = df['z_fMRI'].max() + 0.5
g.set(ylim=(0, 2.2))

g.figure.suptitle('', fontsize=22, y=1.01)  # Add title above plot

g.figure.set_dpi(300)
plt.savefig(f'{plot_dir}/reinstatement_placebodata_gender_effect.png', bbox_inches='tight')
plt.close()
#plt.show()

In [18]:
# placebo data, gender * group
df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
df_tmp.loc[df_tmp['Contrast']==contrasts_CSS, 'CS_Type'] = 'CSS'
df_tmp.loc[df_tmp['Contrast']==contrasts_CSR, 'CS_Type'] = 'CSR'
df = df_tmp.copy()
df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()

df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')

plt.style.use('seaborn-v0_8-poster')

g = sns.catplot(x='group', y='z_fMRI', hue='CS_Type',
                row='roi_name', 
                kind='bar', 
                hue_order=['CSS', 'CSR'],
                errorbar='se',
                height=6, aspect=1,
                data=df)

g.set_axis_labels('', 'z_fMRI')
#g.legend.set_title('')
  
y_min = df['z_fMRI'].min() - 0.5
y_max = df['z_fMRI'].max() + 0.5
g.set(ylim=(0, 2.2))

g.figure.suptitle('', fontsize=22, y=1.01)  # Add title above plot

g.figure.set_dpi(300)
plt.savefig(f'{plot_dir}/reinstatement_placebodata_group_effect.png', bbox_inches='tight')
plt.close()
#plt.show()

In [19]:
for roi in df_current['roi_name'].unique():
    print(f'{roi}')
    df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Contrast'].isin(contrasts_of_interest))].copy()
    df_tmp.loc[df_tmp['Contrast'] == contrasts_CSS, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast'] == contrasts_CSR, 'CS_Type'] = 'CSR'
    df = df_tmp.copy()
    df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    
    df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')
    
    plt.style.use('seaborn-v0_8-poster')
    
    g = sns.catplot(x='Drug', y='z_fMRI', hue='CS_Type',
                    order=['Placebo', 'Oxytocin'], 
                    col='group', 
                    row='Gender', 
                    kind='bar', 
                    hue_order=['CSS', 'CSR'],
                    errorbar='se',
                    height=6, aspect=1,
                    data=df)
    
    g.set_axis_labels('', 'z_fMRI')
    #g.legend.set_title('')
    if g.legend is not None:
        g.legend.set_title('')
    else:
        print(f"Warning: No legend generated for {roi_name}")    
    y_min = df['z_fMRI'].min() - 0.5
    y_max = df['z_fMRI'].max() + 0.5
    g.set(ylim=(0, 2.2))
    
    g.figure.suptitle(f'{roi} in Extinction', fontsize=22, y=1.01)  # Add title above plot
    
    g.figure.set_dpi(300)
    plt.savefig(f'{plot_dir}/reinstatement_plot_{roi}.png', bbox_inches='tight')
    #plt.show()
    plt.close()

lh_hippocampus
rh_ACC
lh_amygdala
lh_ACC
rh_vmpfc
lh_insula
rh_amygdala
rh_insula
lh_vmpfc
rh_hippocampus
